In [1]:
from bs4 import BeautifulSoup
import requests
# Parsing and creating xml data
from lxml import etree as et

# Store data as a csv file written out
from csv import writer

# In general to use with timing our function calls to Indeed
import time

# Assist with creating incremental timing for our scraping to seem more human
from time import sleep

# Dataframe stuff
import pandas as pd

# Random integer for more realistic timing for clicks, buttons and searches during scraping
from random import randint

# Multi Threading
import threading

# Threading:
from concurrent.futures import ThreadPoolExecutor, wait
import math
import mysql.connector
from datetime import date
import json

In [2]:
import selenium

# Check version I am running
selenium.__version__

'4.15.2'

In [3]:
import undetected_chromedriver as uc

In [4]:
from selenium import webdriver

# Starting/Stopping Driver: can specify ports or location but not remote access
from selenium.webdriver.chrome.service import Service as ChromeService

# Manages Binaries needed for WebDriver without installing anything directly
from webdriver_manager.chrome import ChromeDriverManager

In [5]:
# Allows searchs similar to beautiful soup: find_all
from selenium.webdriver.common.by import By

# Try to establish wait times for the page to load
from selenium.webdriver.support.ui import WebDriverWait

# Wait for specific condition based on defined task: web elements, boolean are examples
from selenium.webdriver.support import expected_conditions as EC

# Used for keyboard movements, up/down, left/right,delete, etc
from selenium.webdriver.common.keys import Keys

# Locate elements on page and throw error if they do not exist
from selenium.common.exceptions import NoSuchElementException

In [6]:
response = requests.get(
  url='https://headers.scrapeops.io/v1/browser-headers',
  params={
      'api_key': 'cca4ced0-490d-41a0-b258-46f2ad7e74b3',
      'num_results': '100'}
)
header_browser_list = response.json()
print(header_browser_list['result'][0])

{'upgrade-insecure-requests': '1', 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36', 'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8', 'sec-ch-ua': 'Google Chrome;v="86", "Chromium";v="86", ";Not A Brand";v="99"', 'sec-ch-ua-mobile': '?0', 'sec-ch-ua-platform': 'macOS', 'sec-fetch-site': 'none', 'sec-fetch-mod': '', 'sec-fetch-user': '?1', 'accept-encoding': 'gzip, deflate', 'accept-language': 'en-US,en;q=0.7'}


In [7]:
response = requests.get(
  url='https://headers.scrapeops.io/v1/user-agents',
  params={
      'api_key': 'cca4ced0-490d-41a0-b258-46f2ad7e74b3',
      'num_results': '100'}
)
user_agent_list = response.json()
print(user_agent_list['result'][0])

Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36


In [8]:
random_index_user_agent = randint(0, len(user_agent_list)-1)
random_index_header_browser = randint(0, len(header_browser_list)-1)
user_agent_random = user_agent_list['result'][random_index_user_agent]
header_browser_random = header_browser_list['result'][random_index_header_browser]

In [9]:
def create_option():
    random_index_user_agent = randint(0, len(user_agent_list)-1)
    random_index_header_browser = randint(0, len(header_browser_list)-1)
    user_agent_random = user_agent_list['result'][random_index_user_agent]
    header_browser_random = header_browser_list['result'][random_index_header_browser]
    # Allows you to cusotmize: ingonito mode, maximize window size, headless browser, disable certain features, etc
    option = uc.ChromeOptions()

    # Going undercover:
    option.add_argument("--incognito")


    # # Consider this if the application works and you know how it works for speed ups and rendering!

    option.add_argument('--headless=chrome')
    user_agent = user_agent_random
    option.add_argument(f"user-agent={user_agent}")

    # Thêm header vào Options
    headers = header_browser_random

    for key, value in headers.items():
        option.add_argument(f"--header={key}:{value}")
    
    return option

In [10]:
option = create_option()
driver = uc.Chrome(service=ChromeService(ChromeDriverManager().install()),options=option)
driver.implicitly_wait(10)  # Set the implicit wait time to 10 seconds
driver.set_page_load_timeout(30)
driver.get("https://www.topcv.vn/viec-lam-it")

job_count = driver.find_element(By.CSS_SELECTOR,'[class="job-header"] h2 b').text.replace(",", "")
if (int(job_count) % 50 == 0):
    max_page  = int(job_count) / 50
else:
    max_page = math.floor(int(job_count) / 50) + 1
print(max_page)
driver.quit()

54


In [11]:
job_urls = []

In [12]:
driver = uc.Chrome(service=ChromeService(ChromeDriverManager().install()),options=create_option())
for page_number in range(1, int(max_page+1)):
    link = f"https://www.topcv.vn/viec-lam-it?page={page_number}"
    driver.get(link)
    print("Đang xử lý: ", link)
    job_url_list = driver.find_elements(By.CSS_SELECTOR, '.job-item-2 h3[class="title"] a')
    print(job_url_list)
    print("******************************************************************")
    for uu in job_url_list:
        link = uu.get_attribute("href")
        job_urls.append(link)
driver.quit()

Đang xử lý:  https://www.topcv.vn/viec-lam-it?page=1
[<undetected_chromedriver.webelement.WebElement (session="bc564276b00e7eb782d58385a2de04f0", element="f.0497817DF9570F7810503C3DE342802D.d.7464741BD9FC49E4B13E38AA880FBE83.e.299")>, <undetected_chromedriver.webelement.WebElement (session="bc564276b00e7eb782d58385a2de04f0", element="f.0497817DF9570F7810503C3DE342802D.d.7464741BD9FC49E4B13E38AA880FBE83.e.300")>, <undetected_chromedriver.webelement.WebElement (session="bc564276b00e7eb782d58385a2de04f0", element="f.0497817DF9570F7810503C3DE342802D.d.7464741BD9FC49E4B13E38AA880FBE83.e.301")>, <undetected_chromedriver.webelement.WebElement (session="bc564276b00e7eb782d58385a2de04f0", element="f.0497817DF9570F7810503C3DE342802D.d.7464741BD9FC49E4B13E38AA880FBE83.e.302")>, <undetected_chromedriver.webelement.WebElement (session="bc564276b00e7eb782d58385a2de04f0", element="f.0497817DF9570F7810503C3DE342802D.d.7464741BD9FC49E4B13E38AA880FBE83.e.303")>, <undetected_chromedriver.webelement.WebEl

In [13]:
conn = mysql.connector.connect(
    host='103.56.158.31',
    port = 3306,
    user= 'tuyendungUser',
    password='sinhvienBK',
    database= 'ThongTinTuyenDung'
)
cursor = conn.cursor()

In [14]:
sql = 'INSERT IGNORE INTO Stg_ThongTin_raw(Web, Nganh, Link, TenCV, CongTy, TinhThanh, Luong, LoaiHinh, KinhNghiem, CapBac, HanNopCV, YeuCau, MoTa, PhucLoi, SoLuong) VALUES (%s, %s, %s, %s,%s,%s,%s, %s, %s, %s, %s, %s, %s, %s,%s)'

In [15]:
sql_link = 'SELECT Link FROM Stg_ThongTin_raw where Web =\'TopCV\''

In [16]:
cursor.execute(sql_link)
result = cursor.fetchall()
remove_url_list = [row[0] for row in result]

In [17]:
print("Số lượng url cào về: ", len(job_urls))
for job_url in remove_url_list:
    if job_url in job_urls:
        job_urls.remove(job_url)
print("Số lượng url mới lấy được sau xử lý:", len(job_urls))

Số lượng url cào về:  2633
Số lượng url mới lấy được sau xử lý: 2210


In [23]:
driver = uc.Chrome( options=create_option())
if len(job_urls) > 0:
    for job_url in job_urls:
        try:
            if "https://www.topcv.vn/brand/" in job_url:
                continue
            link = job_url
            print("Đang xử lí....", link)
            driver.get(link)
            Web = 'TopCV'
            Nganh = 'IT'
            Link = link
            TenCV = driver.find_element(By.CSS_SELECTOR, '.job-detail__info--title').text
            CongTy = driver.find_element(By.CSS_SELECTOR, '[class="company-name-label"] a').text
            SoLuong = "1"
            YeuCau = ""
            MoTa = ""
            PhucLoi = ""
            for i in range(len(driver.find_elements(By.CSS_SELECTOR, '[class="job-detail__info--section"]'))):
                if "Mức lương" in driver.find_elements(By.CSS_SELECTOR, '[class="job-detail__info--section"]')[i].find_element(By.CSS_SELECTOR, '[class="job-detail__info--section-content-title"]').text:
                    Luong = driver.find_elements(By.CSS_SELECTOR, '[class="job-detail__info--section"]')[i].find_element(By.CSS_SELECTOR, '[class="job-detail__info--section-content-value"]').text
                if "Địa điểm" in driver.find_elements(By.CSS_SELECTOR, '[class="job-detail__info--section"]')[i].find_element(By.CSS_SELECTOR, '[class="job-detail__info--section-content-title"]').text:
                    TinhThanh = driver.find_elements(By.CSS_SELECTOR, '[class="job-detail__info--section"]')[i].find_element(By.CSS_SELECTOR, '[class="job-detail__info--section-content-value"]').text
            try:
                HanNopCV = driver.find_element(By.CSS_SELECTOR, '[class="job-detail__info--deadline"]').text.split(":")[-1].strip()
            except:
                HanNopCV = date.today()
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            List = soup.find_all('div', class_='job-description__item')
            YeuCau = List[1].text
            MoTa = List[0].text
            PhucLoi = List[2].text
            for i in range(len(soup.find_all('div', class_='box-general-group'))):
                if "Số lượng tuyển" in soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-title').text:
                    SoLuong = soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-value').text.split()[0]
                if "Hình thức làm việc" in soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-title').text:
                    LoaiHinh = soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-value').text
                try:
                    if "Kinh nghiệm" in soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-title').text:
                        KinhNghiem = soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-value').text
                except:
                    KinhNghiem = "Không có"
                if "Cấp bậc" in soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-title').text:
                    CapBac = soup.find_all('div', class_='box-general-group')[i].find('div', class_='box-general-group-info-value').text
            cursor.execute(sql, (Web, Nganh, Link, TenCV, CongTy, TinhThanh, Luong, LoaiHinh, KinhNghiem, CapBac, HanNopCV, YeuCau, MoTa, PhucLoi, SoLuong))
            conn.commit()
        except:
            continue
else:
    print("Không có tin mới để thêm.")
driver.quit()
cursor.close()
conn.close()


Đang xử lí.... https://www.topcv.vn/viec-lam/nhan-su-cham-soc-hoc-vien-va-van-hanh-lop-hoc-sro-khong-yeu-cau-kinh-nghiem/1233007.html?ta_source=ITJobs_LinkDetail
Đang xử lí.... https://www.topcv.vn/viec-lam/senior-php-developer-on-site-hybrid-luong-tu-15-25-trieu/1237050.html?ta_source=ITJobs_LinkDetail
Đang xử lí.... https://www.topcv.vn/viec-lam/senior-backend-developer-php/1240027.html?ta_source=ITJobs_LinkDetail
Đang xử lí.... https://www.topcv.vn/viec-lam/fullstack-developer-nodejs-reactjs-tu-4-nam-kinh-nghiem-tai-ha-noi/360276.html?ta_source=ITJobs_LinkDetail
Đang xử lí.... https://www.topcv.vn/viec-lam/nhan-vien-ky-thuat-tai-tp-ha-noi-thu-nhap-tu-7-10-trieu-khong-yeu-cau-kinh-nghiem/1245464.html?ta_source=ITJobs_LinkDetail
Đang xử lí.... https://www.topcv.vn/viec-lam/truong-nhom-digital-marketing-thu-nhap-up-25-trieu-thang-ha-noi/1191290.html?ta_source=ITJobs_LinkDetail
Đang xử lí.... https://www.topcv.vn/viec-lam/chuyen-vien-add-chuyen-vien-phat-trien-ung-dung/1244093.html?ta_s

In [ ]:
print(job_urls)

['https://www.topcv.vn/viec-lam/ky-su-cau-noi-tieng-han-bien-phien-dich-tieng-han-nganh-it-korean-brse/1132735.html?ta_source=ITJobs_LinkDetail', 'https://www.topcv.vn/viec-lam/vfx-artist-junior/1168046.html?ta_source=ITJobs_LinkDetail', 'https://www.topcv.vn/viec-lam/lap-trinh-vien-software-developers-muc-luong-den-35m/1154432.html?ta_source=ITJobs_LinkDetail', 'https://www.topcv.vn/viec-lam/nhan-vien-ky-thuat-may-tinh/1206111.html?ta_source=ITJobs_LinkDetail', 'https://www.topcv.vn/brand/tek-experts/tuyen-dung/it-support-engineers-microsoft-software-support-fresher-experienced-j654470.html?ta_source=ITJobs_LinkDetail', 'https://www.topcv.vn/viec-lam/tester-mobile-app/1016516.html?ta_source=ITJobs_LinkDetail', 'https://www.topcv.vn/viec-lam/chuyen-vien-kiem-thu-phan-mem-mang-chung-khoan/1206536.html?ta_source=ITJobs_LinkDetail', 'https://www.topcv.vn/viec-lam/senior-database-administrator-dba/1206364.html?ta_source=ITJobs_LinkDetail', 'https://www.topcv.vn/viec-lam/android-java-develo